In [1]:
from requests_html import AsyncHTMLSession
from bs4 import BeautifulSoup
import asyncio
import logging

In [2]:
logging.disable(logging.WARNING)

async def fetch_and_parse(edition, article, session, debug=False):
    url = f'https://nordiskfamiljebok.dh.gu.se/article/{edition}/{article}'
    
    response = await session.get(url)
    await response.html.arender(sleep=5) 
    
    if debug:
        print(f"Successfully fetched {'1st' if article == 1 else f'{article}nd'} article from {'1st' if edition == 1 else f'{edition}nd'} edition")

    return response.html

In [3]:
session = AsyncHTMLSession()

ed1_start = 1
ed2_start = 222844

ed1_steps = 0
ed2_steps = 5

ed1_count = 251508
ed2_count = 251508

async with asyncio.TaskGroup() as task_group:
    ed1_tasks = []#[task_group.create_task(fetch_and_parse(1, n, session, True)) for n in range(ed1_start, min(ed1_start + ed1_steps, ed1_count) + 1)]
    ed2_tasks = [task_group.create_task(fetch_and_parse(2, n, session, True)) for n in range(ed2_start, min(ed2_start + ed2_steps, ed2_count) + 1)]

ed1_html = [task.result() for task in ed1_tasks]
ed2_html = [task.result() for task in ed2_tasks]

Successfully fetched 222846nd article from 2nd edition
Successfully fetched 222849nd article from 2nd edition
Successfully fetched 222848nd article from 2nd edition
Successfully fetched 222844nd article from 2nd edition
Successfully fetched 222845nd article from 2nd edition
Successfully fetched 222847nd article from 2nd edition


In [108]:
import re

def segment_article(html):
    soup = BeautifulSoup(html, "html.parser")

    article = soup.find('article', {'data-v-5ad7308b': True})

    if article:
        article = article.decode_contents()

        for match in re.split("[.*]\s*<br\s*/?>\s*<br\s*/?>\s*", article):
            print(match, "\n")
    else:
        print("No matching article found.")

In [109]:
n = 0

soup = BeautifulSoup(ed2_html[n].html, "html.parser")

article = soup.find('article', {'data-v-5ad7308b': True})
content = article.decode_contents()

segment_article(ed2_html[n].html)

<b>Trucksystem,</b> af eng. <i>to truck</i> [tra'k], byta, ett närmast från England hämtadt uttryck för det system, som i allmänna drag innebär, att arbetaren ej betalas i kontanta pengar (resp. checker), utan h. o. h. eller delvis i naturaförmåner, antingen i produkter, vid hvilkas framställande han medverkat -- oafsedt huruvida han har behof däraf eller ej -- eller också -- hvilket är vanligast -- i varor eller anvisningar på varor, som ingå i hans allmänna behof, men för hvilka arbetsgifvaren eller hans medintressent betingat ett mer eller mindre oskäligt högt pris. Ibland innebär systemet, att arbetaren tar emot lönen i kontanta pengar, men med ett uttryckligt eller underförstådt förbehåll, att han skall göra sina inköp i en handelsbod eller en restaurang, som drifves af arbetsgifvaren eller af en honom närastående person. En lindrigare form af trucksystem är den ordning, <br/><br/> då arbetaren tvingas att hyra bostad af arbetsgifvaren och betala alltför hög hyra (det i England my

In [106]:
import unicodedata
import locale

locale.setlocale(locale.LC_COLLATE, 'sv_SE.UTF-8')

def split_with_validation(text):
    """
    Split text using regex and validate segments with a boolean function.
    """
    # Split text using the regex
    segments = re.split(r"\.\s*<br\s*/?>\s*<br\s*/?>", text)

    # Filter segments with the custom boolean function

    last_valid_segment = None # segments[0]
    last_valid_word = None
    valid_segments = []

    for segment in segments:
        segment = unicodedata.normalize('NFC', segment)
        clean_segment = re.sub(r"<.*?>|[-']", "", segment).strip().lower()

        first_word = re.match(r"\b(\w[\w']*)", clean_segment).group(0)  # Match the first word
        first_word_uml = re.sub(r'ii', 'ü', first_word)

        if last_valid_segment:

            if ((locale.strcoll(last_valid_word, first_word) <= 0 or locale.strcoll(last_valid_word, first_word_uml) <= 0)
                 and (re.match(r"[\w'-]+(?:[\s]*[.,\(\[])?", clean_segment))):
                print("OK!", segment)

            else:
                print("NOT OK!", first_word, segment)
                last_valid_segment
        else:
            last_valid_segment = segment
            last_valid_word = first_word

        #print(clean_segment, "\n")

split_with_validation(content)

OK!  Trud (isl. "jöniör, "styrka"), nord. myt., dotter af åskans gud Tor och hans maka, Sif. Jfr A 11 vis
OK!  Trudgelmer, nord. myt. Se Bergelmer
OK!  Trudgeon [traMjen]. Se Simidrott, sp. 565
OK!  Trudhem 1. Trudvang (isl. ^rtäheimr 1. fyruftvangar), nord. myt., namnet på det rike, hvarest Tor härskade och där han hade sin präktiga hall Bilskirner. Th. W.* <br/><br/> Trudoviker (af ry. trud, arbete, möda), beteckning på ett ryskt politiskt parti, som till ett antal af omkr. 80, hufvudsakligen bönder, läkare, lärare och advokater, bildades i den första duman af Aladjin, Anikin, Zjilkin, Bondarev, Sjaposjnikov och Onitsko. Stående till vänster om "kadetterna", ifrade denna grupp för agrariska reformer samt höjande af det kroppsarbetande och intelligenta proletariatets ställning genom allmän rösträtt. Efter deltagande i förhandlingarna i Viborg häktades flera af gruppens ledare. På ett möte i Imatra okt. 1906 utvidgades partiets program, och en centralkommitté tillsattes med 32 filialer

In [97]:
import re

text = "Tru'stis domFnica. Another-word (test), word [example]"

matches = re.findall(r"[\w'-]+(?:[\s]*[.,\(\[])?", text)

print(matches)


["Tru'stis", 'domFnica.', 'Another-word (', 'test', 'word [', 'example']
